In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np

# internal imports
from tgb.utils.utils import *
from tgb.linkproppred.evaluate import Evaluator
from modules.hyper_edgebank import HyperEdgeBankPredictor
from modules.neighbor_loader import LastNeighborLoaderGraphmixer
from modules.early_stopping import  EarlyStopMonitor
from test_hyper_edgebank import test_edgebank
from tgb.linkproppred.dataset_pyg import PyGLinkPropPredDataset, PyGLinkPropPredDatasetHyper
from graphmixer import *

# Initialize data and EdgeBank

In [2]:
# recreate args
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
args = Namespace(dataset='tgbl-hypergraph',
                 lr=1e-4,
                 bs=2000,  # use larger batch size since we're not training
                 k_value=10,
                 num_epoch=100,
                 seed=1,
                 mem_dim=100,
                 time_dim=10,
                 emb_dim=100,
                 tolerance=1e-6,
                 patience=100,
                 num_run=1,
                 wandb=False,
                 bipartite=False,
                 memory_name='static',
                 emb_name='sum',
                 use_inventory=False,
                 debt_penalty=0,
                 consum_rwd=0,
                 gpu=0,
                 num_train_days=-1,
                 num_layers=2,
                 dropout=0.1, 
                 num_neighbors=300,
                 use_prev_sampling=True, # since we tgbl-hypergraph dataset is before change to sampling method
                 time_gap=2000)
device = torch.device("cpu")

In [3]:
# Initialize dataset - copied from run_experiment() in tgnpl.py
with open(os.path.join(PATH_TO_DATASETS, f"{args.dataset.replace('-', '_')}/{args.dataset}_meta.json"), "r") as f:
    metadata = json.load(f)
# set global data variables
num_nodes = len(metadata["id2entity"])  
num_firms = metadata["product_threshold"]
num_products = num_nodes - num_firms              
dataset = PyGLinkPropPredDatasetHyper(name=args.dataset, root="datasets")
metric = dataset.eval_metric
neg_sampler = dataset.negative_sampler
evaluator = Evaluator(name=args.dataset)
data = dataset.get_TemporalData().to(device)
train_loader, val_loader, test_loader = set_up_data(args, data, dataset)

dataset.load_val_ns()  # load validation negative samples

Dataset tgbl-hypergraph url not found, download not supported yet.
file found, skipping download
Dataset directory is  /lfs/turing1/0/zhiyinl/supply-chains/TGB/tgb/datasets/tgbl_hypergraph
loading processed file
Train: N=214674, 273 days; Val: N=44991, 45 days; Test: N=45612, 47 days


In [5]:
# initialize edgebank, fit on train
edgebank = HyperEdgeBankPredictor(num_firms, num_products, consecutive=True)
train_data = data[dataset.train_mask]
edgebank.fit(train_data.src, train_data.dst, train_data.prod)

Fit on 214674 edges; found 38756 unique


In [6]:
test_edgebank(val_loader, neg_sampler, "val", evaluator, metric, edgebank, use_counts=False, use_prev_sampling=args.use_prev_sampling)

  0%|                                                                                                               | 0/23 [00:00<?, ?it/s]/lfs/turing1/0/zhiyinl/supply-chains/TGB/examples/linkproppred/general/test_hyper_edgebank.py:62: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  neg_batch_list = torch.Tensor(neg_batch_list)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:01<00:00, 15.60it/s]


0.3399311602115631

In [7]:
test_edgebank(val_loader, neg_sampler, "val", evaluator, metric, edgebank, use_counts=True, use_prev_sampling=args.use_prev_sampling)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:04<00:00,  5.46it/s]


0.6819248199462891

In [8]:
# try with subset of train
train_days = data.t[dataset.train_mask].unique()  # original set of train days
days_to_keep = train_days[-30:]  # keep train days from the end, since val follows train
new_train_mask = torch.isin(data.t, days_to_keep)
print(new_train_mask.sum())
edgebank.fit(data[new_train_mask].src, data[new_train_mask].dst, data[new_train_mask].prod)

tensor(30157)
Fit on 30157 edges; found 12434 unique


In [9]:
exist_baseline = test_edgebank(val_loader, neg_sampler, "val", evaluator, metric, edgebank, use_counts=False, use_prev_sampling=args.use_prev_sampling)
count_baseline = test_edgebank(val_loader, neg_sampler, "val", evaluator, metric, edgebank, use_counts=True, use_prev_sampling=args.use_prev_sampling)
print(exist_baseline, count_baseline)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:05<00:00,  4.11it/s]

0.4436224400997162 0.6085948348045349


# Try fitting model on tiny data

In [16]:
# first 100 data points in val, NO neighbors
mini_loader = TemporalDataLoader(data[dataset.val_mask][:100], batch_size=args.bs)
assert len(mini_loader) == 1
batch = next(iter(mini_loader))
pos_src, pos_prod, pos_dst, pos_t, pos_msg = batch.src, batch.prod, batch.dst, batch.t, batch.msg
print(pos_src.shape)

torch.Size([100])


In [17]:
# try graphmixer
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, size=args.num_neighbors, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on FIXED negative samples from neg_sampler
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, 
                       opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       neg_sampler=neg_sampler, split_mode="val", 
                       num_firms=num_firms, num_products=num_products, use_prev_sampling=args.use_prev_sampling)
    if epoch % 10 == 0:
        print(epoch, loss)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.07s/it]


10 2.4627556800842285


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.82s/it]


20 0.9582272171974182


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:24<00:00, 24.93s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.64s/it]


980 6.983817729633301e-05


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:26<00:00, 26.18s/it]


990 6.845591269666329e-05


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.00s/it]


In [18]:
# test MRR on same data
# graphmixer: similar result before and after reset
test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products, 
     use_prev_sampling=args.use_prev_sampling)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:21<00:00, 21.57s/it]


1.0

In [19]:
# same results once we reset
neighbor_loader.reset_state()
test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
     use_prev_sampling=args.use_prev_sampling)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.20s/it]


1.0

In [20]:
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, size=args.num_neighbors, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on RANDOM negative samples
# should be able to learn but loss will go down a bit more randomly
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, 
                       opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       ns_samples=6, num_firms=num_firms, num_products=num_products)
    if epoch % 10 == 0:
        print(epoch, loss)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.27s/it]


10 2.5753934383392334


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:25<00:00, 25.76s/it]


20 1.2772018909454346


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:24<00:00, 24.98s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.08s/it]


980 0.06998059153556824


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:22<00:00, 22.03s/it]


990 0.02266303263604641


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:21<00:00, 21.84s/it]


In [21]:
# test MRR on same data
# graphmixer: similar result before and after reset
mrr1 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products, 
            use_prev_sampling=args.use_prev_sampling)

# near perfect result after reset
neighbor_loader.reset_state()
mrr2 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

print(f'MRR: before resetting = {mrr1:.4f}; after resetting = {mrr2:.4f}')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.23s/it]

MRR: before resetting = 0.9700; after resetting = 0.9750


In [11]:
# first 100 data points in val, WITH time-varying neighbors
mini_loader = TemporalDataLoader(data[dataset.val_mask][:100], batch_size=10)
print(len(mini_loader))
batch = next(iter(mini_loader))
pos_src, pos_prod, pos_dst, pos_t, pos_msg = batch.src, batch.prod, batch.dst, batch.t, batch.msg
print(pos_src.shape)

10
torch.Size([10])


In [12]:
# still trying model with static memory
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, size=args.num_neighbors, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on FIXED negative samples from neg_sampler, time-varying neighbors
# results: loss seems quite stable for time-varying neighbors, unlike TGNPL
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       neg_sampler=neg_sampler, split_mode="val", 
                       num_firms=num_firms, num_products=num_products,
                       use_prev_sampling=args.use_prev_sampling)
    if epoch % 10 == 0:
        print(epoch, loss)

  0%|                                                                                                               | 0/10 [00:00<?, ?it/s]/lfs/turing1/0/zhiyinl/supply-chains/TGB/examples/linkproppred/general/graphmixer.py:110: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  neg_batch_list = torch.Tensor(neg_batch_list)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.67s/it]


10 0.07160531124100089


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.67s/it]


20 0.012159588560461999


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.66s/it]


...

140 0.0001422197023202898


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.69s/it]


150 0.00012447159224393545


 40%|█████████████████████████████████████████▏                                                             | 4/10 [00:18<00:27,  4.65s/it]


KeyboardInterrupt: 

In [13]:
# test MRR on same data
# graphmixer: similar result before and after reset
mrr1 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

# after reset, still perfect result
neighbor_loader.reset_state()
mrr2 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

print(f'MRR: before resetting = {mrr1:.4f}; after resetting = {mrr2:.4f}')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:21<00:00,  2.16s/it]

MRR: before resetting = 1.0000; after resetting = 1.0000


In [14]:
# still trying model with static memory
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, size=args.num_neighbors, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on RANDOM negative samples, time-varying neighbors
# should be able to learn but loss will go down a bit more randomly
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       ns_samples=6, num_firms=num_firms, num_products=num_products,
                       use_prev_sampling=args.use_prev_sampling)
    if epoch % 10 == 0:
        print(epoch, loss)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:37<00:00,  3.70s/it]


10 0.1754837293177843


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:35<00:00,  3.55s/it]


20 0.17874257266521454


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:35<00:00,  3.60s/it]


...

860 0.02389266278660216


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:39<00:00,  3.99s/it]


870 0.004887620785098079


 20%|████████████████████▌                                                                                  | 2/10 [00:10<00:42,  5.29s/it]


KeyboardInterrupt: 

In [15]:
# test MRR on same data
# low cross-entropy loss should correspond to high MRR
# if we don't reset neighbor loader, results is ok
mrr1 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

# much better near perfect result after resets
neighbor_loader.reset_state()
mrr2 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

print(f'MRR: before resetting = {mrr1:.4f}; after resetting = {mrr2:.4f}')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:22<00:00,  2.29s/it]

MRR: before resetting = 0.8286; after resetting = 0.9933
